In [1]:
%config IPCompleter.greedy=True

In [2]:
from os import listdir
from pickle import dump
import re
import string
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from pickle import load

Using TensorFlow backend.


In [11]:
def extract_features(directory):
    model = VGG16()
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    print(model.summary())
    features = dict()
    for name in listdir(directory):
        filename = directory+'/'+name
        image = load_img(filename, target_size=(224,224))
        image = img_to_array(image)
        image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        image_id = name.split('.')[0]
        features[image_id]=feature
    return features

In [4]:
directory = './Flicker8k_Dataset'
features = extract_features(directory)
dump(features, open('features.pkl', 'wb'))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

1/1 [==============================] - 0s 331ms/step


1/1 [==============================] - 0s 371ms/step


1/1 [==============================] - 0s 351ms/step


1/1 [==============================] - 0s 312ms/step


...

1/1 [==============================] - 0s 497ms/step


1/1 [==============================] - 0s 329ms/step


1/1 [==============================] - 0s 343ms/step


1/1 [==============================] - 0s 499ms/step


1/1 [==============================] - 0s 491ms/step


In [5]:
def load_doc(fn):
    f=open(fn,'r')
    text=f.read()
    f.close()
    return text

In [6]:
def load_descriptions(doc):
    mapping=dict()
    for line in doc.split('\n'):
        tokens = line.split()
        if len(line)<2:
            continue
        image_id, image_desc = tokens[0], tokens[1:]
        image_id = image_id.split('.')[0]
        image_desc = ' '.join(image_desc)
        if image_id not in mapping:
            mapping[image_id] = list()
        mapping[image_id].append(image_desc)
    return mapping

In [8]:
def clean_descriptions(descriptions):
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            desc = desc.split()
            desc = [word.lower() for word in desc]
            desc = [w.translate(table) for w in desc]
            desc = [word for word in desc if len(word)>1]
            desc = [word for word in desc if word.isalpha()]
            desc_list[i] =  ' '.join(desc)             

In [9]:
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [10]:
doc = load_doc('./Flickr8k_text/Flickr8k.token.txt')
descriptions = load_descriptions(doc)
clean_descriptions(descriptions)
save_descriptions(descriptions, 'descriptions.txt')